This GESIS notebook demonstrates the functionality of [Deepface model](https://github.com/serengil/deepface) to infer the gender of persons using images. It is a supplement to the [GESIS Gender Inference website](http://193.175.238.89/Gender_Inference/images.html), which includes more details, guidelines and limitations that you should consider before using this notebook.

Follow these steps to use the model for your own image or an image url. Upon exiting the notebook, all data will be deleted.<br/>
1. If notebook is not run on the launch, please run all cells by clicking "Cell" -> "Run all" <br/>
2. Upload a photo from your computer <b>OR</b> type a url to an image file on the Web. These options are interchangeable.
3. Click "Predict gender" to run Deepface on your input data. <br/>
4. Button "Clear" ensures no data is stored afterwards.

<b>Important</b>: After clicking "Cell" -> "Run all", If the buttons are not appearing please restart the kernal or refresh the page.

In [ ]:
# Importing necessary libraries
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# Keras outputs warnings using `print` to stderr so let's direct that to devnull temporarily
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import clear_output, Javascript
from PIL import Image
from IPython.display import HTML
from pathlib import Path
from io import BytesIO
import requests as requests
from IPython.display import HTML
from deepface import DeepFace
from deepface.extendedmodels import  Gender
import pathlib
import shutil
import warnings
warnings.filterwarnings('ignore') #hide warnings

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL) #hide info messages

%reload_ext autoreload
%autoreload 2
display(Javascript('IPython.notebook.execute_cells_below()')) #execute cells below

# Load the gender model weights
home = str(Path.home())
source = '../DeepFace/gender_model_weights.h5'
destination = home + '/.deepface/weights/gender_model_weights.h5'
if not pathlib.Path(destination).exists():
    if not pathlib.Path(home + '/.deepface/weights/').exists():
        os.mkdir(home + "/.deepface/")
        os.mkdir(home + "/.deepface/weights/")
    dest = shutil.copyfile(source, destination)

# Initiate a gender model
models = {}
models["gender"] = Gender.loadModel()

<IPython.core.display.Javascript object>

OSError: Unable to open file (file signature not found)

In [ ]:
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>. Or you can use the buttons in the navigation menu.''')

In [ ]:
# running deepface
def deepface():
    try:
        deepface_pred = DeepFace.analyze(filename, models=models, actions=['gender'], enforce_detection=True)
        if deepface_pred['gender'] == 'Woman':
            gender = 'Female'
        if deepface_pred['gender'] == 'Man':
            gender = 'Male'
        print("Predicted gender for your input is: " + gender)
    except:
        print("Ooops! No face has been detected")

In [ ]:
# Set up layout for upload widgets and update the upload counter


def initialize_widgets():
    global upload, link, name, layout
    upload = widgets.FileUpload(
        accept='image/*',
        multiple=False
    )
    link = widgets.Text(
        placeholder='url to an image file',
        description='Link:',
        disabled=False
    )
    input_box = HBox([upload, link])
    layout = VBox([input_box, submit])
    display(layout)
    upload.observe(on_upload_change, names='_counter')


def on_upload_change(b):
    if upload._counter > 1:
        upload._counter = 1
    else:
        upload._counter = upload._counter

    
global clear, submit, submitted
submitted = False
display(HTML('<p>Please either use <b>upload</b> field to select a file from your computer or provide a <b>url</b> to a web image. <br/><b>Important</b>: please use a link to an image file, ending with a common image formats -.jpg, .png.</p>'))
# print('Please either use upload field to select a file from your computer or provide a url to a web image. ')
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()


# Clear the output and delete the files
def on_clear_clicked(b):
    clear_output()
    initialize_widgets()
    for filename in os.listdir('./'):
        if filename.endswith(".jpg") or filename.endswith(".csv"):
            os.remove(filename)
    submitted = False


# convert images. Save the file (is deleted afterwards)
def preprocess_and_save(content):
    global filename
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    filename = "testfile.jpg"
    image.save(filename)
    

# Submitting an image either as fileupload or from url
def on_submit_clicked(b):
    global submitted
    global content
    if upload.value and link.value:   # both inputs are given
        display(HTML('Please choose only one method for input'))
        return False
    elif upload.value:   # file is uploaded
        display(HTML('<b>Please be patient, don\'t restart the kernel or refresh the page. The model may take some time to predict the gender.\n \n</b>'))
        uploaded_filename = next(iter(upload.value))
        content = upload.value[uploaded_filename]['content']
    elif link.value:   # url is fiven
        display(HTML('<b>Please be patient, don\'t restart the kernel or refresh the page. The model may take some time to predict the gender.\n \n</b>'))
        content = requests.get(link.value).content
    else:   # no input is given
        display(HTML('No image has been uploaded'))
        return False
    preprocess_and_save(content)
    if submitted:   # cleaning the previous results
        clear_output()
        initialize_widgets()
    display(HTML('An image you submitted:'))
    display(widgets.Image(value=content, width=200))
    deepface()
    display(clear)
    submitted = True


submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)

In [ ]:
clear_output()
display(HTML('<p>This was a demo version of deepface on one image. You can go to the <a href="deepface_Gender_Prediction_Batch.ipynb" target="_blank">next notebook</a> to apply the model on a batch of images</p>'))